<div style="text-align: center;">

<h2 style= "color:yellowgreen">Fixing the intern's errors
</h2>

</div>
An intern recently started working with ROS and generated the following code to move the turtle in the turtle simulator.
The turtle is supposed to move foward and then backward.
However, the intern only spent 1 day with the code and did not test it because they waited to the last day.
You have taken the intern's place and it's now your job to fix their errors.

Your tasks are:  

A) Debug the given code  

- Find possible errors using the ROS debugging tools  
- Fix them until no errors occur when running  

B) Test the code  

- The turtle should move forward and then backward


Below you can see the intern code. 
The code is located in folder ./ex_workspace/src/intern_pkg/src/move_turtle.py

The following training instructions will get you up to speed with ros debugging and allow you to fix the intern's code.


<div style="text-align: center;">
<h2 style= "color:tomato">Training Instructions</h2>
</div>

## <emph style="color:seagreen">1) Executing the code with rosrun</emph>

<ol> 
<li> Run the following code to generate the VS-Code container</li>

- Ensure that your terminal is located in the ex1 folder by using the `cd` command

In [ ]:
dts code editor --recipe ../ex1-recipe

---

- **Error Note:** If you get an error saying "An SSL key pair needs to be generated first..." then run the following command:

In [ ]:
dts setup mkcert

- **Error Note:** If you get another error saying you need the system library 'libnss3-tools', use the following code to download the library:

In [ ]:
sudo apt install libnss3-tools

---

<li> Open a terminal in the VS-Code container (in the browser). </li>

![vscode](../assets/ex1-images/vscode.png)

- To open a terminal, select the three lines in the top left --> Terminal --> New Terminal 

**Note:** When we write "`open new terminal`", it means a new terminal in the VS-Code container and **not** in your host OS. 



<li> Build and source the packages in the ex_workspace running </li>


In [ ]:
cd ex1/ex_workspace; catkin build; source devel/setup.bash

1. **Workspace Note:** In the VS-Code container, our catkin workspace will always be `ex_workspace`. 
It is important that we run the command `catkin build` from inside the directory. 
The build process should complete within a few seconds and the summary should report that `4` packages were built.

2. **Catkin Note:** When we move into VNC container in the next section and when we use duckiebot, we will use the standard `catkin_ws` instead of ex_workspace.

3. **Source Note:** If everything goes well, this command should return without any errors or output.


## Running ROS package

The intern already created a ROS package. 

It is called the *intern_pkg* package. 
We will see more information about ROS packages in the future. 
For now, let's us run the `intern_pkg`


<ol type="a">
<li>Start ROS. In a terminal, run:</li>


In [ ]:
roscore

Open a `new terminal`, activate the workspace (sourcing), and try to run the intern's code:


In [ ]:
source devel/setup.bash
rosrun intern_pkg turtle_move.py

Yep, it does not work.  
Use the error message to fix the first error.  
The package files can be found in ex1 --> ex_workspace --> src --> intern_pkg --> src.  
Take note of the error you will fix, e.g., add a comment to the modification you have done in the code.  
**Hint:** It looks like the intern forgot to define a variable. 

**Note:** `rosrun` allows you to use the package name to directly run a _node_ from a _package_ (without having to know the package path).

Usage:

    rosrun [package_name] [node_name]

- Once you have fixed the errors, you should be able to run the package using the previous code

You should see printed messages such as the following:

<br><img src="../assets/ex1-images/turtle_move_print.png" style="width: 80%; height: auto;"></br>

</li>

</ol>
</ol>


Great start.  
Once your code is running, we can move on with our debugging.  
However, we need to learn more about some debugging tools in ROS.  
**Note:** You can stop the *intern_pkg* by pressing CTRL+C in the terminal.  


## <emph style="color:seagreen">2) ROS Nodes</emph>

Open up a **new terminal**, and let's use `rosnode` to see what running `roscore` did... Bear in mind to keep the previous terminal open by opening a new terminal.

`rosnode` displays information about the ROS nodes that are currently running. The `rosnode list` command lists these active nodes:


In [ ]:
rosnode list


You will see:

    /rosout

This showed us that there is only one node running: `rosout`. This is always running as it collects and logs nodes' debugging output.

If you are still running the intern_pkg:

    /rosout
    /turtle_move


The `rosnode info` command returns information about a specific node.

    rosnode info /rosout

This gave us some more information about `rosout`, such as the fact that it publishes to the topic `/rosout_agg`.

```bash
------------------------------------------------------------------------
Node [/rosout]
Publications:
    * /rosout_agg [rosgraph_msgs/Log]

Subscriptions:
    * /rosout [unknown type]

Services:
    * /rosout/get_loggers
    * /rosout/set_logger_level

contacting node http://machine_name:54614/ ...
Pid: 5092
```

## Review

What was covered:

- `roscore` = ros+core : master (provides name service for ROS) + rosout (stdout/stderr) + parameter server (parameter server will be introduced later)
- `rosnode` = ros+node : ROS tool to get information about a node.
- `rosrun` = ros+run : runs a node from a given package. 

Now that you understand how ROS nodes work, let's look at how ROS topics work. Also, feel free to press `Ctrl`- `C` to stop running `roscore` or `rosrun`. 


 



## <emph style="color:seagreen">3) ROS Topics</emph>

*ROS topics* help communicate information among ROS nodes.  
For example, the *turtle_move* node is publishing to the topic `/turtle1/cmd_vel`.  
This node is suppose to publish a velocity to the topic for the turtle to move.  
*More about this in a bit*


### Introducing rostopic

The rostopic tool allows you to get information about ROS **topics**.

You can use the help option to get the available sub-commands for rostopic


In [ ]:
rostopic -h




-   ```
    rostopic bw     display bandwidth used by topic
    rostopic echo   print messages to screen
    rostopic hz     display publishing rate of topic    
    rostopic list   print information about active topics
    rostopic pub    publish data to topic
    rostopic type   print topic type
    ```
<ol>

<li>Using rostopic list </li>

rostopic list returns a list of all topics currently subscribed to and published.

**Note:** If you closed roscore and the rosrun turtle_move.py from the intern package, run roscore and the turtle_move node as in part **1**

**In a new terminal, run:**


In [ ]:
rostopic list -v


This displays a verbose list of topics to publish to and subscribe to and their type.
Something like: 

```bash
  Published topics:
    * /rosout [rosgraph_msgs/Log] 2 publishers
    * /rosout_agg [rosgraph_msgs/Log] 1 publisher
    * /turtle1/cmd_vel [geometry_msgs/Twist] 1 publisher
  
  Subscribed topics:
    * /rosout [rosgraph_msgs/Log] 1 subscriber
```

The information in the brackets is the type of message the topic publishes or subscribes. 
We can look at the details of the message using rosmsg:


In [ ]:
rosmsg show geometry_msgs/Twist

<li>Using rostopic echo </li>

rostopic echo shows the data published on a topic.


In [ ]:
rostopic echo [topic]


- Change [topic] to the topic whose data you want to see

**Note:** If you closed roscore and the rosrun turtle_move.py from the intern package, run roscore and the turtle_move node as in part **1**

**In a new terminal, run:**


In [ ]:
rostopic echo turtle1/cmd_vel

- Press CTRL+C to exit out of echo

<p style="text-align: center;">
<strong>Disclaimer:</strong>
This tutorial is an adaptation of the <a href="https://wiki.ros.org/ROS/Tutorials/">Official ROS tutorial</a> originally released under <a href="http://creativecommons.org/licenses/by/3.0/">Creative Commons Attribution 3.0</a> license.
</p>